# Introducing the `PFBPassband.jl` package

A [companion notebook](01_pfb_response.ipynb) describes how to calculate both
the unaliased and aliased frequeuncy response of a *polyphase filter bank*
(PFB).  In this notebook we will explore `PFBPassband.jl`, which provides a high
level interface for calculating the aliased PFB frequency response.  The aliased
PFB frequency response is often referred to a the *PFB passband*, hence the
package name.  As mentioned in the package's [`README.md`](../README.md) file,
which this notebook largely follows, the unaliased frequency response of a PFB
can be obtained by using the [`DSP.jl`](https://docs.juliadsp.org/latest/)
pacakge with the PFB coefficients obtained from `PFBPassband.jl`.  In this
notebook, "frequency response" will generally refer to the unaliased frequency
response while "passband" will generally to refer to the aliased frequency
response.

We will also look at PFB passbands of real world PFBs from the [Green Bank
Telescope](https://en.wikipedia.org/wiki/Green_Bank_Telescope) and the [MeerKAT
Radio Telescope](https://en.wikipedia.org/wiki/MeerKAT).

First we have to setup the environment for this notebook.  This may take some
time (and produce more output) the first time you run it.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
if Pkg.is_manifest_current() != true
    Pkg.develop(path="..") # Needed until package is registered in General
    Pkg.precompile();
end

  Activating project at `~/.julia/dev/PFBPassband/notebooks`


## PFB specifications

### Low level PFB specification

The (unaliased) response of the PFB filter coefficients at a given frequency can
be computed from the coefficients alone (and the given frequency).  To compute
the (aliased) passband of a PFB channel at a given frequency within the channel
we also need to know the number of channels of the PFB so we will know how the
post-decimation power will alias into the PFB channel.  The `PFBPassband.jl`
package provides the `passband(h, nchan, nfine)` which will compute the
(aliases) passband response for `nfine` evenly spaced points across a PFB
channel for the given `nchan` channel PFB with coefficients, `h`.  Here again
the `nchan` value includes both positive and negative frequencies.  This method
computes the PFB channel passband for any PFB whose coefficients are and number
of channels are known.

### High level PFB specification

While specifying a PFB by its filter coefficiients and number of channels is
extremely generalized, it is also possible to specify a PFB by a small number of
parameters and an implicit parameterized implementation.  One example of this,
and the primary/initial focus of the `PFBPassband.jl` package, is the PFB
implementation of the CASPER tool flow.

The CASPER PFB is parameterized by several parameters, including one that
indicates whether to model a known buggy version of the CASPER PFB.
The `PFBPassband.jl` package provides the `CasperPolyphaseFilterbank` structure
that encapsulates these parameters.  As specified in the `README.md` file, these
parameters are:

- Number of channels (`nchan`)
- Number of taps (`ntaps`)
- Normalized channel width (`width`)
- Windowing function (`window`)
- Low pass filter function (`lpf`)
- Bug flag (whether to model bug) (`bug`)

Please refer the `README.md` file and/or the doc string of the
`CasperPolyphaseFilterbank` structure for more information.  The
`PFBPassband.jl` package provides several constructors for thie structure, but
for maximum clarity we will use the one that takes only keywaord arguments.

Other PFB implementations can be added in future versions of `PFBPassband.jl`.

## PFB frequency response (unaliased)

The (unaliased) frequency response of a PFB can be computed by using the
`DSP.jl` pacakge.  This requires obtaining the PFB filter coefficients.  Because
there is no aliasing, the number of PFB channels is not important for the
response itself, but it can be useful to know when interpreting/plotting the
response.

Let's compare the (unaliased) freqeuncy response of two CASPER PFBs that share
common design parameters except for the `width` parameter.  One will have
`width=1` and the other will have `width=7/8`.  We will plot the frequency
response for three adjacent PFB channels using 64 points per coarse channel.

In [7]:
using PFBPassband, DSP, Plots

# Specify common PFB parameters
nchan = 16
ntaps = 8
# Create PFBs
pfb1 = CasperPolyphaseFilterbank(nchan, ntaps, width=1.0)
pfb2 = CasperPolyphaseFilterbank(nchan, ntaps, width=7/8)

# Generate w values at which to plot central channel
nfine = 64
w = range(-2/nchan, step=1/(nchan*nfine), stop=+2/nchan) .* π;

# For thwse small/few PFBs, calling coefs repeatedly isn't a big deal,
# but for larger/more PFBs caching the results can be beneficial.
caefcache = Dict{CasperPolyphaseFilterbank,Vector{Float64}}()

# For each PFB, plot the frequency response for each of the three channels
p = plot(title="PFB width comparison", xlabel="PFB channel", xticks=-1:1, ylabel="dBc")
for c in -1:1
    for (pfb, ls) in ((pfb1, :solid), (pfb2, :dash))
        # Get coefs from cache (and add as needed)
        h = get!(()->coefs(pfb), coefcache, pfb)
        hc = h .* cispi.(2*c.*(0:nchan*ntaps-1)/nchan)
        H = freqresp(PolynomialRatio(hc, [1]), w);
        plot!(p, w*nchan/2π, 10log10.(abs2.(H)); ls, color=c+2, label="chan $c width $(pfb.width)")
    end
end

vline!(p, -1.5:1.5, lc=:gray, ls=:dashdot, primary=false)

UndefVarError: UndefVarError: `coeffcache` not defined